<a href="https://colab.research.google.com/github/SUBRATNAYAK7504/Chat-Q-A-Powered-By-ChatGPT/blob/master/llama_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q llama-index
!pip install -q openai
!pip install transformers
!pip install -q accelerate
!pip install llama-cpp-python
!pip install pypdf
!pip install einops transformers_stream_generator
!pip install -i https://test.pypi.org/simple/ bitsandbytes
!pip install langchain
!pip install sentence_transformers

Looking in indexes: https://test.pypi.org/simple/


In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index import ServiceContext, set_global_service_context

from llama_index.llms import HuggingFaceLLM
from llama_index.prompts import PromptTemplate
from llama_index.prompts.prompts import SimpleInputPrompt
from IPython.display import Markdown, display

In [ ]:
system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."



# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-cr

In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)


In [ ]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [ ]:
#to read the document and create index
doc_path = "./raw_data"
documents = SimpleDirectoryReader(doc_path).load_data()
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("How does the Act define an 'officer who is in default'?")
display(Markdown(f"<b>{response}</b>"))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<b>According to Section 465 of the Companies Act, 2013, an "officer who is in default" is defined as:
"(4) The copies of every notification issued under this section shall, as soon as may be after it has been issued, be laid before each House of Parliament.]
(465) Officer who is in default means any officer of a company who is in default in complying with the provisions of this Act or any rules made thereunder."
Therefore, an officer who is in default refers to any officer of a company who fails to comply with the provisions of the Companies Act, 2013 or the rules made thereunder. This includes directors, managing directors, secretaries, and other officers of the company who are responsible for complying with the Act's provisions.</b>

In [ ]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='0917a768-505b-4f8e-a6e2-61418c178e7c', embedding=None, metadata={'page_label': '213', 'file_name': 'CompaniesAct2013_Without_INdex.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='91474618-3c54-4e3a-ada4-f37430d7a1f8', node_type=None, metadata={'page_label': '213', 'file_name': 'CompaniesAct2013_Without_INdex.pdf'}, hash='872b6b0975609c7b249283f0dc5b50bfa7b9086502036dc1b711b650bec64086'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='548ac410-1c29-4dc5-8454-49560f7968ad', node_type=None, metadata={'page_label': '213', 'file_name': 'CompaniesAct2013_Without_INdex.pdf'}, hash='064878475342a4b367e93a61e71d25a6737860a8643e0d81ac36d53d779f8d6d')}, hash='2ccf018369aaa7d73e17e27c2fb9b12ab688eed7ecb782a29e6b67dbb1650c7e', text='213  (4) The copies of every notification issued under this section shall, as soon as may be after it has been  \nissued,  be laid be